<a href="https://colab.research.google.com/github/audreyakwenye/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import pandas as pd 
import scipy
from scipy import stats

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', sep=',', header=None, skiprows=1, names = ["A1", "A2", "A3", "A4", "A5", "A6",
                                                                                                                                                    "A7", "A8", "A9", "A10", "A11", "A12",
                                                                                                                                                    "A13", "A14", "A15", "A16"])
data.dtypes

data = data[~data['A2'].isin(['?'])]
data = data[~data['A14'].isin(['?'])]

In [0]:
def unique_entries(column):
  return data.groupby(column).count()

unique_entries(data['A1'])

In [31]:
data.A2 = data.A2.astype(float).fillna(0.0)
data.A11 = data.A11.astype(float).fillna(0.0)
data.A14 = data.A14.astype(float).fillna(0.0)
data.A15 = data.A15.astype(float).fillna(0.0)

data.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11    float64
A12     object
A13     object
A14    float64
A15    float64
A16     object
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [32]:
#A16 T Test 
data = data.replace('-', 0)
data = data.replace('+', 1)

data['A16'] = pd.to_numeric(data['A16'])



approved = data.loc[data['A16'] == 1]
denied = data.loc[data['A16'] == 0]

#Null Hypothesis rejected for Approvals 
scipy.stats.ttest_1samp(data['A16'], 0.5)

Ttest_1sampResult(statistic=-2.688202437770095, pvalue=0.0073639117652293026)

In [53]:
scipy.stats.ttest_1samp(approved['A11'], 0.5)

Ttest_1sampResult(statistic=11.345861307855682, pvalue=5.008971776145305e-25)

In [49]:
#Null Hypothesis rejected for denied entries and column 'A11'
scipy.stats.ttest_1samp(denied['A11'], 0.5)

Ttest_1sampResult(statistic=1.4706121629062208, pvalue=0.14225519830616878)

In [78]:
scipy.stats.ttest_1samp(approved['A8'], 0.5)

Ttest_1sampResult(statistic=12.344537799243001, pvalue=1.5393777329385195e-28)

In [79]:
scipy.stats.ttest_1samp(denied['A8'], 0.5)

Ttest_1sampResult(statistic=6.766305077106182, pvalue=5.255988507389747e-11)

In [109]:
from scipy import stats

#Null Hypothesis rejected, there is a correlation between A16 and A7
tab1 = pd.crosstab(data.A7, data.A16, margins=True)

stats.chi2_contingency(tab1)[0:3]

(45.07395755782807, 0.0010782695291895073, 20)

In [110]:
print(tab1)

A16    0    1  All
A7                
?      3    0    3
bb    30   25   55
dd     4    2    6
ff    46    8   54
h     51   87  138
j      5    3    8
n      2    2    4
o      1    1    2
v    223  164  387
z      2    6    8
All  367  298  665


In [107]:
#Null Hypothesis rejected, there is a correlation between A16 and A5
tab2 = pd.crosstab(data.A5, data.A16, margins=True)

stats.chi2_contingency(tab2)[0:3]

(23.06126597566228, 0.0007762165281546467, 6)

In [108]:
print(tab2)

A16    0    1  All
A5                
g    256  251  507
gg     0    2    2
p    111   45  156
All  367  298  665


In [111]:
tab3 = pd.crosstab(data.A4, data.A16, margins=True)

stats.chi2_contingency(tab3)[0:3]

(23.06126597566228, 0.0007762165281546467, 6)

In [112]:
print(tab3)

A16    0    1  All
A4                
l      0    2    2
u    256  251  507
y    111   45  156
All  367  298  665


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

In the T tests, the null hypothesis was rejected for the denied class and column A11 which means the entries fall outside of random chance. Other numeric columns don't show significant connection. 

The chi test shows more coorelation as the null hypothesis was rejected for columns A5 and A7 which show they both have a high correlation with the approved and denied classes. I also saw that the chi test results for A5 and A4 are the same. 